In [3]:
import iotadas as adas
import cv2
import os
import imutils
from tkinter import *
import os
import cv2
from matplotlib import pyplot
import numpy as np
from time import sleep
import threading
import multiprocessing

def check_file_existance(filePath):
    try:
        with open(filePath, 'r') as f:
            return True
    except FileNotFoundError as e:
        return False
    except IOError as e:
        return False
    
def user_image_capturer (user,dataPath):
    if user is not None:
        personName = user
        personPath = dataPath + '/' + personName

        if not os.path.exists(personPath):
            print('Carpeta creada: ',personPath)
            os.makedirs(personPath)

        cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
        
        faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
        count = 0

        while True:

            ret, frame = cap.read()
            if ret == False: break
            frame =  imutils.resize(frame, width=640)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            auxFrame = frame.copy()

            faces = faceClassif.detectMultiScale(gray,1.3,5)

            for (x,y,w,h) in faces:
                cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
                rostro = auxFrame[y:y+h,x:x+w]
                rostro = cv2.resize(rostro,(150,150),interpolation=cv2.INTER_CUBIC)
                cv2.imwrite(personPath + '/rotro_{}.jpg'.format(count),rostro)
                count = count + 1
            cv2.imshow('frame',frame)

            k =  cv2.waitKey(1)
            if k == 27 or count >= 500:
                break

        cap.release()
        cv2.destroyAllWindows()
        return True
    else: 
        return False


def train_user (user,dataPath,modelPath):
    
    peopleList = dataPath+'/'+user #os.listdir(dataPath)
    print('Lista de personas: ', peopleList)
    
    labels = []
    facesData = []
    label = 0

    #for nameDir in peopleList:
    
    personPath = dataPath + '/' + user
    print('Leyendo las imágenes')

    for fileName in os.listdir (personPath):
        print('Rostros: ', personPath + '/' + fileName)
        labels.append(label)
        facesData.append(cv2.imread(personPath+'/'+fileName,0))
        #image = cv2.imread(personPath+'/'+fileName,0)
        #cv2.imshow('image',image)
        #cv2.waitKey(10)
    label = label + 1

    # Métodos para entrenar el reconocedor
    face_recognizer = cv2.face.LBPHFaceRecognizer_create()

    # Entrenando el reconocedor de rostros
    print("Entrenando...")
    face_recognizer.train(facesData, np.array(labels))

    # Almacenando el modelo obtenido
    model = modelPath+user+'_'+'modeloLBPHFace.xml'
    print(" model :",model)
    

    if not os.path.exists(modelPath):
        print('Carpeta creada: ',modelPath)
        os.makedirs(modelPath)
            
    face_recognizer.write(model)
    print(" Usario Registrado...")

def user_detect (user,dataPath,modelPath):
    
    detect = False
    if user is not None:
        imagePaths = os.listdir(dataPath)
        print('imagePaths=',imagePaths)

        #face_recognizer = cv2.face.EigenFaceRecognizer_create()
        #face_recognizer = cv2.face.FisherFaceRecognizer_create()
        face_recognizer = cv2.face.LBPHFaceRecognizer_create()

        # Leyendo el modelo
        #face_recognizer.read('modeloEigenFace.xml')
        #face_recognizer.read('modeloFisherFace.xml')
        model = modelPath+user+'_'+'modeloLBPHFace.xml'
        if check_file_existance (model):
            print (' [*] model:',model)
            face_recognizer.read(model)

            cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
            #cap = cv2.VideoCapture('Video.mp4')

            faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

            while True:
                ret,frame = cap.read()
                if ret == False: break
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                auxFrame = gray.copy()

                faces = faceClassif.detectMultiScale(gray,1.3,5)

                for (x,y,w,h) in faces:
                    rostro = auxFrame[y:y+h,x:x+w]
                    rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
                    result = face_recognizer.predict(rostro)
                    print( ' Predict user:',result)


                    cv2.putText(frame,'{}'.format(result),(x,y-5),1,1.3,(255,0,255),1,cv2.LINE_AA)

                    '''
                    # EigenFaces
                    if result[1] < 5700:
                        cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
                    else:
                        cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)

                    # FisherFace
                    if result[1] < 500:
                        cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
                    else:
                        cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
                    '''
                    # LBPHFace
                    if result[1] < 70:
                        cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
                        detect = True
                    else:
                        cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
                        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
                        detect = False

                cv2.imshow('frame',frame)
                k = cv2.waitKey(1)
                if k == 27:
                    break
                    
            cap.release()
            cv2.destroyAllWindows()      
            return detect
        
        else:
            return detect
    else: 
        return detect  

def reg_user (user):
    dataPath    =  './data/reconocimiento' #directorio almacenamiento fotos usuarios para entrenamiento
    modelPath   =  './model/user/'
    if (user != '') and user_image_capturer (user,dataPath):
        train_user (user,dataPath,modelPath)
        return True
    else:
        return False
    
def login_user (user):
    print('login_user:[',user,']')
    dataPath    =  './data/reconocimiento' #directorio almacenamiento fotos usuarios para entrenamiento
    modelPath   =  './model/user/'
    if ((user != '') and (user is not None)):
        if user_detect (user,dataPath,modelPath):
             return True
        else:
            return False
    else:
        return False
# ----------------------------------------------
#  ---------------- Front-End APPP  ------------
# -----------------------------------------------

def registrar_usuario():
    path_users = "./data/users"
    usuario_info = usuario.get()         # Obetnemos la informacion alamcenada en usuario
    contra_info  = contra.get()          # Obtenemos la informacion almacenada en contra

    if not os.path.exists(path_users):
        print('Carpeta creada: ',path_users)
        os.makedirs(path_users)
    file_user = path_users + usuario_info
    
    
    archivo = open(file_user, "w")    # Abriremos la informacion en modo escritura
    archivo.write(usuario_info + "\n")   # escribimos la info
    archivo.write(contra_info)
    archivo.close()

    #Limpiaremos los text variable
    usuario_entrada.delete(0, END)
    contra_entrada.delete(0, END)

    #Ahora le diremos al usuario que su registro ha sido exitoso
    Label(pantalla1, text = "Registro Convencional Exitoso", fg = "green", font = ("Calibri",11)).pack()
    

#--------------------------- Funcion para almacenar el registro facial --------------------------------------
def registro_facial():
    usuario_img = usuario.get()
    if usuario_img != '' and reg_user (usuario_img):
    
        usuario_entrada.delete(0, END)   #Limpiamos los text variables
        contra_entrada.delete(0, END)
        Label(pantalla1, text = "Registro Facial Exitoso", fg = "green", font = ("Calibri",11)).pack()
        sleep (2)
        # Inicio Sesión mediante bluetooth
        pantalla1.destroy()
    else:
        Label(pantalla1, text = "Error Registro!!", fg = "green", font = ("Calibri",11)).pack()
        Label(pantalla1, text = "compruebe si ha intrducido el usuario", fg = "green", font = ("Calibri",11)).pack()
 #------------------------Crearemos una funcion para asignar al boton registro --------------------------------
def registro():
    global usuario
    global contra  #Globalizamos las variables para usarlas en otras funciones
    global usuario_entrada
    global contra_entrada
    global pantalla1
    
    pantalla1 = Toplevel(pantalla) #Esta pantalla es de un nivel superior a la principal
    
    pantalla1.title("Registro")
    pantalla1.geometry("500x450")   #Creamos la ventana
    #--------- Empezaremos a crear las entradas ----------------------------------------
    
    usuario = StringVar()
    contra = StringVar()
    
    Label(pantalla1, text = "Registro facial: debe de asignar un usuario:").pack()
    #Label(pantalla1, text = "").pack()  #Dejamos un poco de espacio
    Label(pantalla1, text = "Registro tradicional: debe asignar usuario y contraseña:").pack()
    Label(pantalla1, text = "").pack()  #Dejamos un poco de espacio
    Label(pantalla1, text = "Usuario * ").pack()  #Mostramos en la pantalla 1 el usuario
    usuario_entrada = Entry(pantalla1, textvariable = usuario) #Creamos un text variable para que el usuario ingrese la info
    usuario_entrada.pack()
    Label(pantalla1, text = "Contraseña * ").pack()  #Mostramos en la pantalla 1 la contraseña
    contra_entrada = Entry(pantalla1, textvariable = contra) #Creamos un text variable para que el usuario ingrese la contra
    contra_entrada.pack()
    Label(pantalla1, text = "").pack()  #Dejamos un espacio para la creacion del boton
    Button(pantalla1, text = "Registro Tradicional", width = 15, height = 1, command = registrar_usuario).pack()  #Creamos el boton

    #------------ Vamos a crear el boton para hacer el registro facial --------------------
    Label(pantalla1, text = "").pack()
    Button(pantalla1, text = "Registro Facial", width = 15, height = 1, command = registro_facial).pack()
    # create button to implement destroy()
    Label(pantalla1,text = "").pack()  #Creamos el espacio  
    Button(pantalla1, text="Quit", command=pantalla1.destroy).pack()
#------------------------------------------- Funcion para verificar los datos ingresados al login ------------------------------------
    
def verificacion_login():
    path_users  = "./data/users"
    log_usuario = verificacion_usuario.get()
    log_contra  = verificacion_contra.get()

    usuario_entrada2.delete(0, END)
    contra_entrada2.delete(0, END)

    lista_archivos = os.listdir (path_users)   #Vamos a importar la lista de archivos con la libreria os
    if log_usuario in lista_archivos:          #Comparamos los archivos con el que nos interesa
        try:
            pt_user  = path_users + "/" + log_usuario
            archivo2 = open(pt_user, "r")  #Abrimos el archivo en modo lectura
            verificacion = archivo2.read().splitlines()  #leera las lineas dentro del archivo ignorando el resto
            if log_contra in verificacion:
                print("Inicio de sesion exitoso")
                Label(pantalla2, text = "Inicio de Sesion Exitoso", fg = "green", font = ("Calibri",11)).pack()
                         # ------------------- Inicio de la app -----------------
                usuario_entrada2.delete(0, END)   #Limpiamos los text variables
                contra_entrada2.delete(0, END)
                sleep(5)
                pantalla2.destroy()
                init_app ()
            else:
                print("Contraseña incorrecta, ingrese de nuevo")
                Label(pantalla2, text = "Contraseña Incorrecta", fg = "red", font = ("Calibri",11)).pack()
        except:
            print("Directorio de Usuario no encontrado")
            Label(pantalla2, text = "Directorio de Usuario no encontrado", fg = "red", font = ("Calibri",11)).pack()
    else:
        print("Usuario no encontrado")
        Label(pantalla2, text = "Usuario no encontrado", fg = "red", font = ("Calibri",11)).pack()
    
#--------------------------Funcion para el Login Facial --------------------------------------------------------
def login_facial ():
    usuario_login = verificacion_usuario.get()    #Con esta variable vamos a guardar la foto pero con otro nombre para no sobreescribir
    print('login_facial ():',usuario_login)
    if login_user (usuario_login):
        print("Inicio de sesion exitoso")
        Label(pantalla2, text = "Inicio de Sesion Exitoso", fg = "green", font = ("Calibri",11)).pack()
         # ------------------- Inicio de la app -----------------
        usuario_entrada2.delete(0, END)   #Limpiamos los text variables
        contra_entrada2.delete(0, END)
        sleep(5)
        pantalla2.destroy()
        init_app ()
    else:
        print("Usuario no encontrado")
        Label(pantalla2, text = "Usuario no encontrado", fg = "red", font = ("Calibri",11)).pack()
        Label(pantalla2, text = "Compruebe e usuario introducido", fg = "red", font = ("Calibri",11)).pack()

#------------------------Funcion que asignaremos al boton login -------------------------------------------------
        
def login ():
    global pantalla2
    global verificacion_usuario
    global verificacion_contra
    global usuario_entrada2
    global contra_entrada2
    
    pantalla2 = Toplevel(pantalla)
    pantalla2.title("Login")
    pantalla2.geometry("500x450")   #Creamos la ventana
    Label(pantalla2, text = "Login facial: debe de asignar un usuario:").pack()
    Label(pantalla2, text = "Login tradicional: debe asignar usuario y contraseña:").pack()
    Label(pantalla2, text = "").pack()  #Dejamos un poco de espacio
    
    verificacion_usuario = StringVar()
    verificacion_contra = StringVar()
    
    #---------------------------------- Ingresamos los datos --------------------------
    Label(pantalla2, text = "Usuario * ").pack()
    usuario_entrada2 = Entry(pantalla2, textvariable = verificacion_usuario)
    usuario_entrada2.pack()
    Label(pantalla2, text = "Contraseña * ").pack()
    contra_entrada2 = Entry(pantalla2, textvariable = verificacion_contra)
    contra_entrada2.pack()
    Label(pantalla2, text = "").pack()
    Button(pantalla2, text = "Inicio de Sesion Tradicional", width = 20, height = 1, command = verificacion_login).pack()

    #------------ Vamos a crear el boton para hacer el login facial --------------------
    Label(pantalla2, text = "").pack()
    Button(pantalla2, text = "Inicio de Sesion Facial", width = 20, height = 1, command = login_facial).pack()
    # create button to implement destroy()
    Label(pantalla2,text = "").pack()  #Creamos el espacio  
    Button(pantalla2, text="Quit", command=pantalla2.destroy).pack()  
#------------------------- Funcion de nuestra pantalla principal ------------------------------------------------

def pantalla_principal ():
    global pantalla          #Globalizamos la variable para usarla en otras funciones
    global butt_login
    global but_reg
    
    pantalla = Tk()
    pantalla.geometry("600x450")  #Asignamos el tamaño de la ventana 
    pantalla.title("Proyecto IOT GCD -UV 2022/23")       #Asignamos el titulo de la pantalla
    Label(text = "APP Login", bg = "gray", width = "300", height = "2", font = ("Verdana", 13)).pack() #Asignamos caracteristicas de la ventana
    
#------------------------- Vamos a Crear los Botones  si tenemos conexión comunicación UART para Bluetooth------------------------------------------------------
    if adas.connect_uart ():
        Label(text = "").pack()  #Creamos el espacio entre el titulo y el primer boton
        butt_login = Button(text = "Iniciar Sesion", height = "2", width = "30", command = login)
        butt_login.pack()
        Label(text = "").pack() #Creamos el espacio entre el primer boton y el segundo boton
        but_reg = Button(text = "Registro", height = "2", width = "30", command = registro) 
        but_reg.pack()
        # create button to implement destroy()
        Label(text = "").pack()  #Creamos el espacio  
        Button(text="Exit", command=exit_app).pack()
    else:
        Label(text = "No se ha podido establecer la comunicación UART/bluetooth", fg = "red", font = ("Calibri",11)).pack()
  
    pantalla.mainloop()

def exit_app ():
    print("exit_app ()")
    print ("if send_uart (4,4):")
    #Enviamos para que se desconecte d HiveMqtt y topic
    
    if adas.send_uart (4,4):
        print("Desconexion Hivemqtt cmpletada!!")
    else:
        print ("Problemas en la desconexión HiveMqtt")
    
    adas.disconnect_uart ()
    print(adas.isConnect())
    if adas.isConnect():
        adas.disconnect_mqtt ()
    if 'sub_server' in globals():
        sub_server.join ()
    if 'r_somno' in globals():
        event.set()
        r_somno.join ()
        
    pantalla.destroy ()
#------------------------- Función de inicio APP : Enviamos por ------------------------------------------------------
def send_app_run ():
    return True

def ini_adas ():
    adas.init_subs_app ()
def init_app ():
    global sub_server
    global r_somno
    global event
    # Enviamos que se conecte a HiveMQTT y arranque la app
    
    if adas.connect_uart () and adas.send_uart (4,2):
        #Limpiamos los botones
        print("init_app ()")
        butt_login.pack_forget()
        but_reg.pack_forget()
        sleep(1)
        # Enviamo mensaje por bluetooth a la pynq y cuando llegue la respuesta si es ok inicio la parte servidora
        if send_app_run ():
            print("init")
            event = multiprocessing.Event() 
            r_somno = threading.Thread(target = adas.run_drowsiness,args=(event,))
            r_somno.start ()
            #adas.init_subs_app ()
            sub_server = threading.Thread(target = ini_adas,args=())
            sub_server.start ()
            print("Estoy esperando en el topic para procesar lso mensajes entrantes")
        else:
            print ("Error al cargar la app. Error conexión HiveMQTT or threading run!!!!")
    else:
        print ("Error al cargar la app. Error comunicación UART!!!")
#----------------------------------------------------
pantalla_principal ()

def connect_uart ():
def disconnect_uart ():
COM5  is open True
Device conectado: COM5 
Dump de configuración:
 Serial<id=0x1d0ed7eb2b0, open=True>(port='COM5', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=1, xonxoff=False, rtscts=False, dsrdtr=False) 

###############################################

uart_connect: True
done serial Connect
Inicio de sesion exitoso
def connect_uart ():
def disconnect_uart ():
El Puerto serie estaba abierto, voy a cerarlo primero
COM5  is open True
Device conectado: COM5 
Dump de configuración:
 Serial<id=0x1d0ed7ebfd0, open=True>(port='COM5', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=1, xonxoff=False, rtscts=False, dsrdtr=False) 

###############################################

uart_connect: True
done serial Connect
def send_uart (ledRGB = 4, color = 4)
str_tx: [4,2]


Retorno da serial: 
init_app ()
init
Voy a conectar el cliente publicador de eventos de la cam
Estoy esperando en el topic para procesar lso mensajes entra

2023-01-07 00:28:46,646 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.32633840472781217
2023-01-07 00:28:46,648 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.11691547071823753
2023-01-07 00:28:46,695 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.313382678096647
2023-01-07 00:28:46,696 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.13040397245529342
2023-01-07 00:28:46,733 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.3185649299076274
2023-01-07 00:28:46,734 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.10628679156203412
2023-01-07 00:28:46,769 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.29615371051685574
2023-01-07 00:28:46,771 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.09738290958997983
2023-01-07 00:28:46,816 root            CRITICAL event:INFO,Blinks:0,COUNTER:0,EAR:0.3219951772634747
2023-01-07 00:28:46,817 root            CRITICAL event:INFO,Yawning:0

 Thread 19312 - Processing on_message
{"clientid": "Client_01", "action": "DTC", "event": "DTDRINK"}
Servidor::>>> str_msg
Servidor:::>>>>> Message received ADASJG01/Server/ from topic {"clientid": "Client_01", "action": "DTC", "event": "DTDRINK"}

 Servidor::>>> main - server: <paho.mqtt.client.Client object at 0x000001D0DE95B850>
Servidor::>>> main - topic: ADASJG01/Pynq/
Servidor::>>> main - msg_in: {"clientid": "Client_01", "action": "DTC", "event": "DTDRINK"}
		 ---> {"clientid": "Client_01", "action": "DTC", "event": "DTDRINK"}
str_msg
event DTDRINK


2023-01-07 00:28:50,272 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.3082627569804516
2023-01-07 00:28:50,273 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.09727308333468228
2023-01-07 00:28:50,316 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.30277825347871157
2023-01-07 00:28:50,317 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.08114373286137122
2023-01-07 00:28:50,361 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.314775980187718
2023-01-07 00:28:50,362 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.08740692589432444
2023-01-07 00:28:50,398 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.29813302958471244
2023-01-07 00:28:50,399 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.10113900272980197
2023-01-07 00:28:50,437 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.3176218341500111
2023-01-07 00:28:50,438 root            CRITICAL event:INFO,Yawning:0

Thread:6180-EVENT:DTDRINK-TIME:2.0226638317108154


...

Thread:11388-EVENT:DTSMOKING-TIME:2.0137007236480713


2023-01-07 00:29:17,452 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.05691378236412815
2023-01-07 00:29:17,486 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.2845982142857143
2023-01-07 00:29:17,487 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.03260869565217391
2023-01-07 00:29:17,523 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.2798416822253226
2023-01-07 00:29:17,524 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.03556472460805307
2023-01-07 00:29:17,567 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.2851540870383799
2023-01-07 00:29:17,568 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.025677311685192595
2023-01-07 00:29:17,610 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.2851540870383799
2023-01-07 00:29:17,611 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.025677311685192595
2023-01-07 00:29:17,656 root            CRITICAL event:INFO,Blink

exit_app ()
if send_uart (4,4):
def send_uart (ledRGB = 4, color = 4)
str_tx: [4,4]



2023-01-07 00:29:25,742 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.3448880914900807
2023-01-07 00:29:25,743 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.13644811993325817
2023-01-07 00:29:25,788 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.3285504300451909
2023-01-07 00:29:25,789 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.12496133929831962
2023-01-07 00:29:25,833 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.3260889222534916
2023-01-07 00:29:25,834 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.15605993852680186
2023-01-07 00:29:25,878 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.33631743482632925
2023-01-07 00:29:25,878 root            CRITICAL event:INFO,Yawning:0,mCOUNTER:0,mar:0.13538728893193186
2023-01-07 00:29:25,913 root            CRITICAL event:INFO,Blinks:1,COUNTER:0,EAR:0.32369017060844496
2023-01-07 00:29:25,914 root            CRITICAL event:INFO,Yawning:


Retorno da serial: 
Desconexion Hivemqtt cmpletada!!
def disconnect_uart ():
El Puerto serie estaba abierto, voy a cerarlo primero
isConnect ()
True
isConnect ()
